In [1]:
%load_ext autoreload
%autoreload 2

from gluonts.mx import Trainer
from gluonts.evaluation import make_evaluation_predictions
from gluonts.model import deepar
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from pathlib import Path
import numpy as np
import pickle
import os
import json
import pandas as pd
import mxnet as mx

mx.random.seed(42)

C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\gluonts\json.py:45: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
DATA_PATH = '../../data/'

In [3]:
ste = pd.read_pickle(f'{DATA_PATH}processed/ste_processed.pkl')
cal = pd.read_pickle(f'{DATA_PATH}processed/cal_processed.pkl')

In [4]:
MODEL_LOCATION = "../../models/"
MODEL_NAME = "deepar_305"

In [6]:
sampled_ids = np.load(f'{DATA_PATH}/interim/sampled_ids.npy').tolist()

In [8]:
wd = os.getcwd()
os.chdir('../data_preparation')
from functions import CreateListDatasets
os.chdir(wd)

train, test, cardinality = CreateListDatasets(ste.loc[sampled_ids, :],
                                              cal,
                                              exclude=['id'])

In [9]:
del ste, cal

In [10]:
trainer = Trainer(epochs=200, 
                  num_batches_per_epoch=50, 
                  learning_rate=0.001, 
                  hybridize=True)

DeepAR_estimator = deepar.DeepAREstimator(
    freq="D", 
    prediction_length=28, 
    context_length=2*28,
    num_layers=2,
    num_cells=40,
    dropout_rate=0.1,
    trainer=trainer,
    distr_output=NegativeBinomialOutput(), 
    use_feat_static_cat=True,
    cardinality=cardinality, 
    use_feat_dynamic_real=True,
    batch_size=30,
)

In [11]:
%%time

DeepAR_predictor = DeepAR_estimator.train(training_data=train)

os.makedirs(MODEL_LOCATION + MODEL_NAME, exist_ok=True)
DeepAR_predictor.serialize(Path(MODEL_LOCATION + MODEL_NAME))

C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\gluonts\dataset\common.py:323: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(string, freq=freq)
C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\gluonts\dataset\common.py:326: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if isinstance(timestamp.freq, Tick):
C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\gluonts\dataset\common.py:328: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  timestamp.floor(timestamp.freq), timestamp.freq
C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\gluonts\dataset\common.py:327: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  return pd.Timestamp(
C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\gluonts\transform\split.py:36: FutureWa

Early stopping based on learning rate scheduler callback (min_lr was reached).


Wall time: 11min 31s


In [12]:
from gluonts.model.predictor import Predictor
deepar_predictor = Predictor.deserialize(
    Path(MODEL_LOCATION + MODEL_NAME))

In [13]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test,  
    predictor=deepar_predictor,  
    num_samples=1000,  
)

In [14]:
%%time

forecasts = list(forecast_it)
tss = list(ts_it)

C:\Users\sebas\AppData\Roaming\Python\Python39\site-packages\gluonts\transform\feature.py:352: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._min_time_point, self._max_time_point, freq=start.freq


Wall time: 3min 39s


In [15]:
os.makedirs(MODEL_LOCATION + MODEL_NAME + '_predictions', exist_ok=True)

with open(MODEL_LOCATION + MODEL_NAME + '_predictions/forecasts.pkl', 'wb') as f:
    pickle.dump(forecasts, f)

with open(MODEL_LOCATION + MODEL_NAME + '_predictions/tss.pkl', 'wb') as f:
    pickle.dump(tss, f)

In [ ]:
# %%time
# with open(MODEL_LOCATION + MODEL_NAME + '_predictions/forecasts.pkl', 'rb') as f:
#     forecasts = pickle.load(f)

# with open(MODEL_LOCATION + MODEL_NAME + '_predictions/tss.pkl', 'rb') as f:
#     tss = pickle.load(f)